In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import scipy.stats as st
import matplotlib.colors as cl

from tqdm.notebook import tqdm

### Fix directories

data_dir = "../data/initial database/output" #path to project folder

os.chdir(data_dir) #Select the project directory

### Import the data from the csv files

def select_directory(sample, file_name = "compounds", part = 1):
    """Returns the file directory corresponding to the file_name.csv file in the folder corresponding to the given sample
    (DiAcids, Fallopia, Ruthenium or Ruthenium2).
    
    the input part indicates the subfolder in which it is stored (pt1 or pt2)."""
    return data_dir + "/" + sample + "/pt" + str(part) + "/" + file_name + ".csv"

def df_from_csv(directory):
    """Extracts the csv file of the file contained in the directory and stores it in a pandas dataframe."""
    with open(directory) as file:
        return pd.read_csv(file)

#Convert the data into pandas dataframes and compute intratios.

sample_names = ["DiAcids", "Fallopia", "Ruthenium", "Ruthenium2"]

data = pd.DataFrame({"compounds" : [df_from_csv(select_directory(sample)) for sample in sample_names],
                     "peaks" : [df_from_csv(select_directory(sample, file_name = "ms2_peaks")) for sample in sample_names],
                     "spectra" : [df_from_csv(select_directory(sample, file_name = "ms2_spectra")) for sample in sample_names]
                    }, index = sample_names)

def peaks_in_list(sample):
    """Returns a dataframe indexed by spectrum_id with entries lists of mz, list of
    intensities and sum of intensities of the ms2 peaks corresponding to this spectrum id."""
    
    groups = data["peaks"][sample].groupby(["spectrum_id"])
    df = groups.agg(list)
    
    return df.loc[:,["mz", "intensity"]]

list_format_sp = pd.Series([peaks_in_list(sample) for sample in sample_names], index = sample_names)

def get_spectrum(sample, spectrum_id):
    """Select a spectrum in the list_format_sp dataframe."""
    return pd.Series(list_format_sp[sample].intensity[spectrum_id], index = list_format_sp[sample].mz[spectrum_id])

def display(spectrum):
    x = spectrum.index
    plt.bar(x, spectrum, width = .5)
    plt.yscale("log")
    
def make_table(sample, spectrum_id):
    """Make a contingency table out of the two spectra sp1 and sp2. Only consider peaks that are in both spectra.
    
    Input should be in the following form
    
    sample : a couple of sample names ("Fallopia", "DiAcids", "Ruthenium" or "Ruthenium2") ;
    spectrum_id : a couple of spectrum_id of the corresponding sample.
    
    The output is a 2 * d contingency tables containing the intensities of the selected peaks, where d is the number of selected peaks in each spectrum.    
    """
    
    #Extract the spectra as table with two columns, mz and intensity.
    sp1 = pd.DataFrame({"mz" : list_format_sp[sample[0]]["mz"][spectrum_id[0]],
                        "intensity" : list_format_sp[sample[0]]["intensity"][spectrum_id[0]]
                       })
    
    sp2 = pd.DataFrame({"mz" : list_format_sp[sample[1]]["mz"][spectrum_id[1]],
                        "intensity" : list_format_sp[sample[1]]["intensity"][spectrum_id[1]]
                       })
    
    #Initialization of the rows of the output table.
    row1, row2 = [], []
    
    k1, k2 = len(sp1), len(sp2)
    
    #Indices running through the mz list.
    i, j = 0, 0
    while i < k1 and j < k2:
        
        mz1, mz2 = sp1.mz[i], sp2.mz[j]
        if abs(mz1 - mz2) < 1e-5 * (mz1 + mz2) / 2: #Machine precision is 10ppm.
            #if the peaks have similar mz, their intensities are stored in row1 and row2.
            row1.append(sp1.intensity[i])
            row2.append(sp2.intensity[j])
            i += 1
            j += 1
        
        #If the mz are distinct, the index corresponding to the smallest mz moves on to the next one (the mz are sorted in incresasing order in the sp_i.mz lists)
        elif sp1.mz[i] < sp2.mz[j]:
            i += 1
            
        else :
            j += 1
            
    return np.array([row1, row2])



In [3]:
pd.set_option("display.max_rows", 200)

#Rename the dataframe to work with.
peaks = data["peaks"]["Fallopia"]
spectra = data["spectra"]["Fallopia"]
compounds = data["compounds"]["Fallopia"]


compounds.index = compounds["compound_id"]

spid_count_by_cid = spectra.groupby("compound_id")["spectrum_id"].count()
all_cid = spid_count_by_cid[spid_count_by_cid > 1].index
nb_cid = len(all_cid) #This is 794. There are 841 compound ids but only 794 have at least two spectra.

#First separate the compounds into a training set of size 320 and an evaluating set of size 80.
#This needs to be done so that there is enough pairs of compound id with the same molecular weight in the evaluation set.

#Create a data frame with columns compound id and parentmz rounded to the third decimal, sorted by parentmz.
spectra_by_cid = pd.DataFrame((1000 * spectra.groupby("compound_id")["parent_mz"].first().sort_values()).apply(int))
spectra_by_cid["compound_id"] = spectra_by_cid.index

#Aggregate the compound id with same parent mz in lists and select the lists of length > 1.
cid_by_pmz = spectra_by_cid.groupby("parent_mz")["compound_id"].agg(list)
comparable_cid = cid_by_pmz[np.array([len(x) for x in cid_by_pmz]) > 1]

len_comparable = len(comparable_cid)
comparable_cid.index = range(len_comparable)

#Select at random some of those list, amounting to around 280 cid (~ 400/794 of the comparable compounds id).
nb_of_lists = 50
nb_of_compounds = 0
while nb_of_compounds < 279:
    seed = 3177606399
    np.random.seed(seed)
    lst_of_indexes = np.random.choice(len_comparable, nb_of_lists, replace = False)
    nb_of_compounds = sum([len(x) for x in comparable_cid[lst_of_indexes]])
    nb_of_lists += 1

#From those ~280 cid, select at random ~ 56 of them
nb_of_lists = 10
nb_of_compounds = 0
while nb_of_compounds < 55:
    nb_of_lists += 1
    lst_of_eval_indexes = lst_of_indexes[:nb_of_lists] #lst_of_indexes is already randomly shuffled.
    nb_of_compounds = sum([len(x) for x in comparable_cid[lst_of_eval_indexes]])
    
eval_cid_flat = []
for i in lst_of_eval_indexes:
    eval_cid_flat += comparable_cid[i] #Evaluating members of the comparable cid.

training_cid_flat = []
for i in lst_of_indexes[nb_of_lists:]:
    training_cid_flat += comparable_cid[i] #Training members of the comparable cid.


#Now, add non-comparable indexes in both the training and the evaluating sets, to a total set of 400 compound ids
#Divided into 80 % for training and 20 % for evaluating, with fraction of comparable cid in each subset roughly equal to the
#fraction of comparable cid in the total set.

comparable_cid_flat = []
for lst in comparable_cid:
    comparable_cid_flat += lst

not_comparable_cid = np.array(all_cid)[np.logical_not([x in comparable_cid_flat for x in all_cid])]


seed = 2312570194
np.random.seed(seed)
random_not_comparable_cid = list(np.random.choice(not_comparable_cid,
                                                  400 - len(training_cid_flat) - len(eval_cid_flat),
                                                  replace = False
                                                 ))
missing_in_eval = 80 - len(eval_cid_flat)
eval_cid_flat += random_not_comparable_cid[:missing_in_eval] #Complete the evaluation set to 80 compound ids.
training_cid_flat += random_not_comparable_cid[missing_in_eval:] #Complete the training set to 320.

#Recap :
#eval_cid_flat is the list of compound ids in the evaluating set
#training_cid_flat is the list of compound ids in the training set

#comparable_cid is a series containing lists of cid having the same molecular weight.
#lst_of_eval_indexes is the list of indexes of comparables cid corresponding of the evaluating set for power testing
#Note : power testing is basically evaluating type 2 error, ie the proportion of false negative. Evaluating it can only be done
#on data labelled as negative, which here is spectra from different molecules having the same molecular weight. This is why
#preparing an evaluating set containing a fair amount of pairs of comparable compound ids is important (comparable means having
#the same molecular weight).

In [4]:
folder_directory = "../data/experiments/experiment 1/"


#Getting the tables back from the csv files.
training = pd.read_csv(folder_directory + "training_info.csv")
eval_alpha = pd.read_csv(folder_directory + "eval_alpha_info.csv")
eval_beta = pd.read_csv(folder_directory + "eval_beta_info.csv")

df = pd.read_csv(folder_directory + "training_tables.csv")
flat_tables = df.groupby("tb_index")["intensity"].agg(list).apply(np.array)
tables = [tb.reshape(2, len(tb) // 2) for tb in flat_tables]
nb_col = [len(tb[0]) for tb in tables]

training["tables"] = tables
training["nb_col"] = nb_col

df = pd.read_csv(folder_directory + "eval_alpha.csv")
flat_tables = df.groupby("tb_index")["intensity"].agg(list).apply(np.array)
tables = [tb.reshape(2, len(tb) // 2) for tb in flat_tables]
nb_col = [len(tb[0]) for tb in tables]

eval_alpha["tables"] = tables
eval_alpha["nb_col"] = nb_col

df = pd.read_csv(folder_directory + "eval_beta.csv")
flat_tables = df.groupby("tb_index")["intensity"].agg(list).apply(np.array)
tables = [tb.reshape(2, len(tb) // 2) for tb in flat_tables]
nb_col = [len(tb[0]) for tb in tables]

eval_beta["tables"] = tables
eval_beta["nb_col"] = nb_col

training_beta = pd.read_csv(folder_directory + "training_beta_info.csv")

df = pd.read_csv(folder_directory + "training_tables_beta.csv")
flat_tables = df.groupby("tb_index")["intensity"].agg(list).apply(np.array)
tables = [tb.reshape(2, len(tb) // 2) for tb in flat_tables]
nb_col = [len(tb[0]) for tb in tables]

training_beta["tables"] = tables
training_beta["nb_col"] = nb_col

The first goal is to estimate phi1

Taking the training tables beta, for each group of compound id (i.e. an element of comparable cid not flat which has been
selected for training), select one table by spectrum id with this compound id and the right number of columns and read the
value....




I didn't have the time to make a code for this during the internship, but I made a plan.

First : estimate $\varphi_1$. In order to do this, compute for each family of molecules an estimation of the covariance matrix of the parameters of the prior Dirichlet distribution.

Let's fix a number of columns $d$.

If compounds 1 to $K$ have the same molecular weight, then the spectra of compound $k$ with $d$ columns provide estimators of the numbers $p_{k, 1}, \dots, p_{k, d}$ drawed under the prior Dirichlet distributaion and associated to the compound $k$.

A moments estimator for the simplex parameter of the prior Dirichlet distribution is given by

$$\hat{\theta}_i = \frac{1}{K} \sum_{k = 1}^K p_{k, i} \, ;$$

and the coefficients of the covariance matrix can then be estimated without bias by

$$\hat{c}_{i, j} = \frac{1}{K-1} \sum_{k = 1}^K (p_{k, i} - \hat{\theta}_{i}) (p_{k, j} - \hat{\theta}_{j}) \, .$$

Under the Dirichlet model, the covariance matrix is given by

$$c_{i, j} = (\delta_{i, j} \theta_i - \theta_i \theta_j) \frac{1}{1 + \varphi_1^{-1}} \, .$$

By doing a linear regression, the coefficient $\frac{1}{1 + \varphi_1^{-1}}$ can be estimated, and this gives a method to estimate $\varphi_1$.

Now we would like to estimate $\varphi_0$ I suggest to do it in the same way in the training tables (for alpha). This time, this is the spectra which are indexed by $k$ and the covariance is that of a Dirichlet-multinomial distribution instead. It could be envisoined to use exactly the same method by saying that the normalized intensities $X_i / N$ follow a Dirichlet distribution of parameter $\varphi_0 \mathbf{p}$ (which is almost true, the error being introduced by the multinomial distribution which has low variance for large N).

Now that we have estimated $\varphi_1$ and $\varphi_0$, it only remains to compute the test statistic, which expresses as

$$T = \frac{1}{B(\varphi_1^{-1} \theta)} \left( \int_{\Delta} \prod_{j = 1}^d \frac{p_j^{\varphi_1^{-1} \theta_j - 1}}{B(n_{1, j}, \varphi_0^{-1}p_j}\mathrm{d}\, p \right) \left( \int_{\Delta} \prod_{j=1}^d \frac{p_j^{\varphi_1^{-1} \theta_j - 1}}{B(n_{2, j}, \varphi_0^{-1}p_j)} \mathrm{d} \, p \right) \left(\int_{\Delta} \prod_{j=1}^d \frac{p_j^{\varphi_1^{-1} \theta_j - 1}}{B(n_{1, j}, \varphi_0^{-1}p_j) B(n_{2, j}, \varphi_0^{-1}p_j)} \mathrm{d} \, p \right)^{-1} \, .$$

These integrals are complicated and probably not easy to calculate even with a computer. If odeint isn't able to perform well with this expression, it could be considered to compute the integrals by a Monte Carlo method and use importance sampling to make it faster.